## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


# 0. Helper Functions

In [1]:
def compose_image_arr(img_list,max_columns,title_list=[],resize_factor = 0.4):
    
    # Check if is enough images to complete the last img composition line and add blank image if needed
    if len(img_list) % max_columns > 0:
        blk = np.copy(img)*0
        for i in range(max_columns - (len(img_list) % max_columns)):
            img_list.append(blk)
            title_list.append("")

    img_list_2d=[]

    for i in range(0,len(img_list)):
        # Check if its not a colored image and stack it like a 3 channel color image
        if len(img_list[i].shape) == 2:
            img_list[i] = np.dstack((img_list[i], img_list[i], img_list[i]))
            
            # if its a binary, then scale to 255
            if np.max(img_list[i]) == 1:
                img_list[i] = img_list[i]*255
        
        # Add image name on the top left corner
        if len(title_list)>0:
            cv2.putText(img_list[i],title_list[i],(10,40),cv2.FONT_HERSHEY_SIMPLEX,1.5,(0,0,255),3,cv2.LINE_AA)

        # if it is the first image of a line, add an empty list to be populated next with following images
        if (i % max_columns) == 0:
            img_list_2d.append([])
        img_list_2d[int(i/max_columns)].append(img_list[i])

    # Concatenate images making a composition of fixed number of images in width
    composed_img = cv2.vconcat([cv2.hconcat(im_list_h) for im_list_h in img_list_2d])
    
    # Resize whole composition
    composed_img_resized = cv2.resize(composed_img, (int(composed_img.shape[1]*resize_factor),int(composed_img.shape[0]*resize_factor)), interpolation = cv2.INTER_AREA)
    
    return composed_img_resized

In [2]:
# Global variables
hls_threshold=(160, 255)
sobel_threshold=(15, 100)

# 1. Compute the camera calibration matrix and distortion coefficients using a set of chessboard images.

In [3]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
    # cv2.imshow('img',img)
    # cv2.waitKey(100)

cv2.destroyAllWindows()

# Get calibration image shape
cal_img = cv2.imread(images[0])
cal_img_shape = cal_img.shape[1::-1]

# Calculate camera calibration params
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, cal_img_shape, None, None)

print("mtx:\n"+str(mtx))
print("dist:\n"+str(dist))

mtx:
[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128938e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist:
[[-0.24688775 -0.02373132 -0.00109842  0.00035108 -0.00258571]]


## 1.1. Define `cal_undistort` function to apply a distortion correction to given images with parameters previously calculated

In [4]:
# Returns undistorted image using the camera intrinsic and extrinsic parameters previously calculated
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## 1.2. Test camera calibration and distortion correction with a chessboard image

In [5]:
# Read an chessboard image
img = mpimg.imread(images[9])

# Undistort image
undistorted = cal_undistort(img, mtx, dist)

# Plot original image and the undistorted image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.0)

# plt.savefig("output_images/camera_calibration.png")

# 2. Color and Gradient Threshold

## 2.1 Define `cal_threshold` function to create a thresholded binary image by applying color transform to HLS colorspace, gradient in x with Sobel operator and threshold combination.

In [6]:
# Return the combined binary image of Sobel and S Channel thresholded
def cal_threshold(img,hls_threshold=(170, 255),sobel_threshold=(20, 100),color_space="RGB"):

    # Convert to HLS color space and separate the V channel
    if color_space == "RGB":
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    else:
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobel_threshold[0]) & (scaled_sobel <= sobel_threshold[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= hls_threshold[0]) & (s_channel <= hls_threshold[1])] = 1

    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary,sxbinary,s_binary,h_channel,l_channel,s_channel

## 2.2. Dynamicly Calibrate `cal_threshold()` 

In [7]:
# Make a list of test images
test_images = glob.glob('test_images/*.jpg')
test_images_index = 0

# Create a window with trackbars and callback functions bolow to handle when values are changed
cv2.namedWindow("Threshold Calibration")

def on_hls_th_l(val):
    global hls_threshold
    hls_threshold = (val,hls_threshold[1])

def on_hls_th_u(val):
    global hls_threshold
    hls_threshold = (hls_threshold[0],val)

def on_sobel_th_l(val):
    global sobel_threshold
    sobel_threshold = (val,sobel_threshold[1])

def on_sobel_th_u(val):
    global sobel_threshold
    sobel_threshold = (sobel_threshold[0],val)

cv2.createTrackbar("Color Threshold Lower","Threshold Calibration",hls_threshold[0],255,on_hls_th_l)
cv2.createTrackbar("Color Threshold Upper","Threshold Calibration",hls_threshold[1],255,on_hls_th_u)
cv2.createTrackbar("Sobel Threshold Lower","Threshold Calibration",sobel_threshold[0],255,on_sobel_th_l)
cv2.createTrackbar("Sobel Threshold Upper","Threshold Calibration",sobel_threshold[1],255,on_sobel_th_u)

while True:
    img_list=[]

    # Read image of current index
    img = cv2.imread(test_images[test_images_index])

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    img_und_thresholded,sxbinary,s_binary,h_channel,l_channel,s_channel = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")

    img_list.append(img)
    img_list.append(img_undistorted)
    img_list.append(np.zeros_like(img))
    img_list.append(h_channel)
    img_list.append(l_channel)
    img_list.append(s_channel)
    img_list.append(sxbinary)
    img_list.append(s_binary)
    img_list.append(img_und_thresholded)

    title_list = ["[{}]Original".format(test_images[test_images_index].split('/')[1]),"Undistorted","","H Channel","L Channel","S Channel","Sobel Threshold","S Channel Threshold","Combined Threshold"]
    img_compostion = compose_image_arr(img_list,3,title_list=title_list,resize_factor=0.40)

    cv2.imshow("Threshold Calibration",img_compostion)
    key = cv2.waitKey(200) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('w'):
        test_images_index += 1
        if(test_images_index >= len(test_images)):
            test_images_index = 0
    elif key == ord('e'):
        test_images_index -= 1
        if(test_images_index < 0):
            test_images_index = len(test_images)-1
    elif key == ord('s'):
        cv2.imwrite("output_images/image_thresholds_cv2.png",img_compostion)

cv2.destroyAllWindows()

print("Calibrated threshold params:")
print("  hls_threshold: ({},{})".format(hls_threshold[0],hls_threshold[1]))
print("  sobel_threshold: ({},{})".format(sobel_threshold[0],sobel_threshold[1]))

Calibrated threshold params:
  hls_threshold: (160,255)
  sobel_threshold: (15,100)


## 2.2. Test `cal_threshold()` with test images and compare original with undistorted images processed

In [8]:
# Read an chessboard image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

# Apply color and gradient threshold
img_thresholded = cal_threshold(img)[0]
img_und_thresholded = cal_threshold(img_undistorted)[0]

# Plot original image, the undistorted image and the respectives thresholded images
f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(12, 11))
# f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_thresholded, cmap='gray')
ax3.set_title('Thresholded Image', fontsize=15)
ax4.imshow(img_und_thresholded, cmap='gray')
ax4.set_title('Thresholded Undistorted Image', fontsize=15)

# Check differences between the original and undistorted binary thresholds
diff_binary = np.zeros_like(img_thresholded)
diff_binary[((img_thresholded == 0) & (img_und_thresholded == 1)) | ((img_thresholded == 1) & (img_und_thresholded == 0))] = 1
# diff_binary[(img_thresholded == 0) & (img_und_thresholded == 1)] = 1
color_diff_binary = np.dstack(( img_thresholded,img_und_thresholded,np.zeros_like(img_und_thresholded))) * 255

ax5.imshow(diff_binary, cmap='gray')
ax5.set_title('Binary diff Image', fontsize=15)
ax6.imshow(color_diff_binary)
ax6.set_title('Colored diff Image', fontsize=15)

plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# plt.savefig("output_images/image_thresholds.png")

# 3. Perspective transform to rectify binary image ("birds-eye view").

## 3.1. Define `cal_perspective()` to return a perspective transform of the lane region

In [9]:
def cal_perspective(img_d,src,dst):
    
    img_size = None
    if len(img_d.shape) > 2:
        img_size = img_d.shape[1::-1]
    else:
        img_size = img_d.shape[::-1]
        img_d = img_d*255
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img_d, M, img_size)

    return warped

## 3.2. Define source and destination vertices to calibrate `cal_perspective()` with a straight lane image

In [10]:

# Auxiliar variables to compute source and destination vertices to be used by cal_perspective()
# These vertices represent the lane region of interest, assuming that the camera is at the center of the vehicle
src_roi_upper = 450
src_roi_lower = img_undistorted.shape[0]

src_horizontal_center_x = img_undistorted.shape[1]/2 +13
src_horizontal_offset_upper = 47
src_horizontal_offset_lower = 470
src_horizontal_drift_upper_l = -15
src_horizontal_drift_upper_r = -11
src_horizontal_drift_lower_l = 0
src_horizontal_drift_lower_r = 0

dst_horizontal_offset = 360

src_vertices = np.array(
   [[src_horizontal_center_x - src_horizontal_offset_lower + src_horizontal_drift_lower_l , src_roi_lower],
    [src_horizontal_center_x - src_horizontal_offset_upper + src_horizontal_drift_upper_l , src_roi_upper], 
    [src_horizontal_center_x + src_horizontal_offset_upper + src_horizontal_drift_upper_r , src_roi_upper], 
    [src_horizontal_center_x + src_horizontal_offset_lower + src_horizontal_drift_lower_r , src_roi_lower]],
    np.float32)

# Define destination vertices to warp
dst_vertices = np.array(
   [[src_horizontal_center_x - dst_horizontal_offset, img_undistorted.shape[0]],
    [src_horizontal_center_x - dst_horizontal_offset, 0],
    [src_horizontal_center_x + dst_horizontal_offset, 0], 
    [src_horizontal_center_x + dst_horizontal_offset, img_undistorted.shape[0]]],
    np.float32)

In [11]:
# Read a straight lane image
img = mpimg.imread("test_images/straight_lines1.jpg")
img2 = mpimg.imread("test_images/straight_lines2.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)
img_undistorted2 = cal_undistort(img2, mtx, dist)

# Apply perspective transform to the undistorted image
img_warped = cal_perspective(img_undistorted,src_vertices,dst_vertices)
img_warped2 = cal_perspective(img_undistorted2,src_vertices,dst_vertices)


for i in range(0,len(src_vertices)):
    pos1 = (int(src_vertices[i][0]),int(src_vertices[i][1]))
    pos2 = (int(src_vertices[i-1][0]),int(src_vertices[i-1][1]))
    cv2.line(img_undistorted,pos1,pos2,(255,150,150),4)
    cv2.line(img_undistorted2,pos1,pos2,(255,150,150),4)
    cv2.circle(img_undistorted,pos1,5,(255,0,0),-1)
    cv2.circle(img_undistorted2,pos1,5,(255,0,0),-1)

for dv in dst_vertices:
    pos = (int(dv[0]),int(dv[1]))
    cv2.circle(img_undistorted,pos,10,(0,255,0),-1)
    cv2.circle(img_undistorted2,pos,10,(0,255,0),-1)

cv2.line(img_warped,(330,0),(330,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped,(980,0),(980,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped2,(330,0),(330,img_warped.shape[0]),(255,0,0),8)
cv2.line(img_warped2,(980,0),(980,img_warped.shape[0]),(255,0,0),8)

# Plot original image and the undistorted image
f, ((ax1, ax2,ax3),(ax4, ax5,ax6)) = plt.subplots(2, 3, figsize=(24, 9))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_warped)
ax3.set_title('Warped Image', fontsize=15)
ax4.imshow(img2)
ax4.set_title('Original Image', fontsize=15)
ax5.imshow(img_undistorted2)
ax5.set_title('Undistorted Image', fontsize=15)
ax6.imshow(img_warped2)
ax6.set_title('Warped Image', fontsize=15)
plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.0)

# plt.savefig("output_images/perspective_transform.png")

## 3.3. Apply a perspective transform to rectify binary image from thresholding step to get the "birds-eye view" image

In [12]:
# Read a straight lane image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

# Apply color and gradient threshold
img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="RGB")[0]

# Source and destination vertices previously defined
# Apply perspective transform to the undistorted image
img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

# Draw vertices
for i in range(0,len(src_vertices)):
    pos1 = (int(src_vertices[i][0]),int(src_vertices[i][1]))
    pos2 = (int(src_vertices[i-1][0]),int(src_vertices[i-1][1]))
    cv2.line(img_undistorted,pos1,pos2,(255,150,150),3)
    cv2.circle(img_undistorted,pos1,5,(255,0,0),-1)

img_warped = np.dstack((img_warped,img_warped,img_warped))

# Plot original image and the undistorted image
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 9))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=15)
ax2.imshow(img_undistorted)
ax2.set_title('Undistorted Image', fontsize=15)
ax3.imshow(img_und_thresholded, cmap='gray')
ax3.set_title('Thresholded Undistorted Image', fontsize=15)
ax4.imshow(img_warped)
ax4.set_title('Warped Image', fontsize=15)
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

# plt.savefig("output_images/perspective_transform.png")

## 4. Detect lane pixels and fit to find the lane boundary

We are going to implement the Sliding window method to find the lane lines and fit a polinomial line to them

## 4.1 Calculate the histogram of all the columns in the lower half of the warped image and get the left and right side peak

In [13]:
# Read a straight lane image
img = mpimg.imread("test_images/test1.jpg")

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

# Apply color and gradient threshold
img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="RGB")[0]

# Source and destination vertices previously defined
# Apply perspective transform to the undistorted image
img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

# Create histogram of image binary activations
histogram = np.sum(img_warped[img_warped.shape[0]//2:,:], axis=0)

# Find the peak of the left and right halves of the histogram
midpoint = np.int(histogram.shape[0]//2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Visualize the resulting histogram
plt.plot(histogram)

## 4. Detect lane pixels and fit to find the lane boundary.

## 4.1 Define common global variables and helper functions

In [14]:
# HYPERPARAMETERS
# Choose the number of sliding windows
nwindows = 9
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# The margin width around the previous polynomial to search
margin_sa = 100

# Moving average length
moving_avg_max_count = 5

# Lanes polynomial Fit variables array
left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

In [15]:
# Reset lanes polynomial fit average arrays
def reset_ma():
    global left_fit_avg_arr
    global right_fit_avg_arr
    left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
    right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

# Return both lane lines sides array without empty values
def get_fit_avg_arr_filtered():
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    return left_fit_avg_arr_filtered, right_fit_avg_arr_filtered

## 4.2.1 Define `cal_sliding_window()` function

In [16]:
def cal_sliding_window(img_warped,nwindows=9,margin=100,minpix=50,draw=True):

    # Take a histogram of the bottom half of the image
    histogram = np.sum(img_warped[img_warped.shape[0]//2:,:], axis=0)

    # Create an output image to draw on and visualize the result
    img_binary_out = np.dstack((img_warped, img_warped, img_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(img_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img_warped.shape[0] - (window+1)*window_height
        win_y_high = img_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        if draw:
            cv2.rectangle(img_binary_out,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(img_binary_out,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fit, right_fit, left_fitx, right_fitx, fity = cal_fit_polynomial(img_warped.shape[0],leftx,lefty,rightx,righty,clear_ma=True)

    if draw:
        # Color in left and right line pixels
        img_binary_out[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        img_binary_out[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Draw the left and right polynomials on top of the img_binary_out image
        if len(left_fitx) > 0 and len(right_fitx) > 0 :
            lane_l = np.dstack((left_fitx, fity))[0]
            lane_r = np.dstack((right_fitx, fity))[0]
            cv2.polylines(img_binary_out,np.int32([lane_l]),False,(0,255,255),thickness=3)
            cv2.polylines(img_binary_out,np.int32([lane_r]),False,(0,255,255),thickness=3)

    return left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out


## 4.2.2 Define `cal_search_around_poly()` function

In [17]:
def cal_search_around_poly(img_warped, margin_sa=100, draw=True):

    # Create an image to draw on and an image to show the selection window
    img_binary_out = np.dstack((img_warped, img_warped, img_warped))*255

    # Grab activated pixels
    nonzero = img_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Calculate the currente lane lines fit averages
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    left_fit_avg = np.array([])
    right_fit_avg = np.array([])

    if len(left_fit_avg_arr_filtered)>0 and len(right_fit_avg_arr_filtered)>0:
        left_fit_avg = np.mean(left_fit_avg_arr_filtered,axis=0)
        right_fit_avg = np.mean(right_fit_avg_arr_filtered,axis=0)
    
    # If any average arrays is empty, then return with empty values. This might be an error durting calibration
    else:
        return left_fit_avg, right_fit_avg, np.array([]), np.array([]), np.array([]), img_binary_out

    # Set the area of search based on activated x-values
    left_lane_inds = ((nonzerox > (left_fit_avg[0]*(nonzeroy**2) + left_fit_avg[1]*nonzeroy + 
                    left_fit_avg[2] - margin_sa)) & (nonzerox < (left_fit_avg[0]*(nonzeroy**2) + 
                    left_fit_avg[1]*nonzeroy + left_fit_avg[2] + margin_sa)))
    right_lane_inds = ((nonzerox > (right_fit_avg[0]*(nonzeroy**2) + right_fit_avg[1]*nonzeroy + 
                    right_fit_avg[2] - margin_sa)) & (nonzerox < (right_fit_avg[0]*(nonzeroy**2) + 
                    right_fit_avg[1]*nonzeroy + right_fit_avg[2] + margin_sa)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Calculate the new fit polynomial
    left_fit, right_fit, left_fitx, right_fitx, fity = cal_fit_polynomial(img_warped.shape[0],leftx,lefty,rightx,righty)

    if draw:
        window_img = np.zeros_like(img_binary_out)

        # Color in left and right line pixels
        img_binary_out[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        img_binary_out[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin_sa, fity]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin_sa, 
                                fity])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin_sa, fity]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin_sa, 
                                fity])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        img_binary_out = cv2.addWeighted(img_binary_out, 1, window_img, 0.3, 0)

        # Draw the left and right polynomials on top of the img_binary_out image
        if len(left_fitx) > 0 and len(right_fitx) > 0 :
            lane_l = np.dstack((left_fitx, fity))[0]
            lane_r = np.dstack((right_fitx, fity))[0]
            cv2.polylines(img_binary_out,np.int32([lane_l]),False,(0,255,255),thickness=3)
            cv2.polylines(img_binary_out,np.int32([lane_r]),False,(0,255,255),thickness=3)
    
    
    return left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out

## 4.2.3 Define `cal_fit_polynomial()` function

In [18]:
def cal_fit_polynomial(img_size_y, leftx, lefty, rightx, righty, clear_ma=False):
    global left_fit_avg_arr
    global right_fit_avg_arr

    # Reset lanes polynomial fit average arrays if requested
    if clear_ma:
        left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
        right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

    # Calculate new fit polynomial values
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
    except TypeError:
        left_fit = []
    
    try:
        right_fit = np.polyfit(righty, rightx, 2)
    except TypeError:
        right_fit = []

    # Append the new value to the moving average array
    left_fit_avg_arr.append(left_fit) 
    right_fit_avg_arr.append(right_fit) 

    # # Append the new value to the moving average array
    # left_fit_avg_arr.append(left_fit) 
    # right_fit_avg_arr.append(right_fit) 

    # left_fit = np.polyfit(lefty, leftx, 2)
    # right_fit = np.polyfit(righty, rightx, 2)
    # if len(left_fit)>0:
    #     left_fit_avg_arr.append(left_fit) 

    # if len(right_fit)>0:
    #     right_fit_avg_arr.append(right_fit) 

    # Remove first (older) value and trim array to max moving average elements
    left_fit_avg_arr = left_fit_avg_arr[1:moving_avg_max_count+1]
    right_fit_avg_arr = right_fit_avg_arr[1:moving_avg_max_count+1]
    
    left_fit_avg_arr_filtered = [ arr for arr in left_fit_avg_arr if len(arr)>0 ]
    right_fit_avg_arr_filtered = [ arr for arr in right_fit_avg_arr if len(arr)>0 ]

    lanes_fit = [left_fit_avg_arr_filtered,right_fit_avg_arr_filtered]
    lanes_fit_avg = [np.array([]),np.array([])]
    lanes_fitx = [np.array([]),np.array([])]

    # Generate x and y values for plotting
    fity = np.linspace(0, img_size_y-1, img_size_y)

    # Calculate average fit and x values for each lane side
    for i in range(len(lanes_fit)):
        if len(lanes_fit[i]) > 0:
            try:
                lanes_fit_avg[i] = np.mean(lanes_fit[i],axis=0)
                lanes_fitx[i] = lanes_fit_avg[i][0]*fity**2 + lanes_fit_avg[i][1]*fity + lanes_fit_avg[i][2]
            except TypeError:
                print('cal_fit_polynomial(): Failed to fit {} line!'.format('left' if i == 0 else 'right'))
                lanes_fitx[i] = np.array([])

    return lanes_fit_avg[0], lanes_fit_avg[1], lanes_fitx[0], lanes_fitx[1], fity


## 4.3.1. Test with single test image

In [19]:
# change to next image index and reset moving average
test_images_index += 1 if test_images_index < (len(test_images)-1) else -(len(test_images)-1)
reset_ma()

In [20]:
# Read image of current index
img = cv2.imread(test_images[test_images_index])

# Undistort image
img_undistorted = cal_undistort(img, mtx, dist)

img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")[0]

# Apply perspective transform to the undistorted thresholded image
img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

left_fit_avg_arr_filtered, right_fit_avg_arr_filtered = get_fit_avg_arr_filtered()

# If both lane lines fit array arent empty, then search around
if len(left_fit_avg_arr_filtered) > 0 and len(right_fit_avg_arr_filtered) > 0 :
    left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out = cal_search_around_poly(img_warped, margin_sa=margin_sa)
else:
    # Calculate the Sliding window method to find lane lines 
    left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

plt.imshow(img_binary_out)

## 4.3.2. Dynamic calibrate parameters for `cal_sliding_window()`, `cal_search_around()` and `cal_fit_polynomial()`

In [21]:
# Make a list of test images
test_images = sorted(glob.glob('test_images/*.jpg'))
test_images_index = 0

# Create a window with trackbars and callback functions bolow to handle when values are changed
cv2.namedWindow("Polynomial Fit")

# Threshold trackbars
def on_hls_th_l(val):
    global hls_threshold
    hls_threshold = (val,hls_threshold[1])

def on_hls_th_u(val):
    global hls_threshold
    hls_threshold = (hls_threshold[0],val)

def on_sobel_th_l(val):
    global sobel_threshold
    sobel_threshold = (val,sobel_threshold[1])

def on_sobel_th_u(val):
    global sobel_threshold
    sobel_threshold = (sobel_threshold[0],val)

cv2.createTrackbar("S Channel Threshold Lower","Polynomial Fit",hls_threshold[0],255,on_hls_th_l)
cv2.createTrackbar("S Channel Threshold Upper","Polynomial Fit",hls_threshold[1],255,on_hls_th_u)
cv2.createTrackbar("Sobel Threshold Lower","Polynomial Fit",sobel_threshold[0],255,on_sobel_th_l)
cv2.createTrackbar("Sobel Threshold Upper","Polynomial Fit",sobel_threshold[1],255,on_sobel_th_u)

# Warp perspective Trackbar
def on_warp_dst_x_offset(val):
    global dst_vertices
    global dst_horizontal_offset
    dst_horizontal_offset = val

    # Redefine destination vertices to warp
    dst_vertices = np.array(
        [[src_horizontal_center_x - dst_horizontal_offset, img_undistorted.shape[0]],
            [src_horizontal_center_x - dst_horizontal_offset, 0],
            [src_horizontal_center_x + dst_horizontal_offset, 0], 
            [src_horizontal_center_x + dst_horizontal_offset, img_undistorted.shape[0]]],
            np.float32)

cv2.createTrackbar("Dst Horizontal Offset","Polynomial Fit",dst_horizontal_offset,600,on_warp_dst_x_offset)

# Polinomial Fit Trackbars
def on_nwindows(val):
    global nwindows
    nwindows = val

def on_margin(val):
    global margin
    margin = val

def on_minpix(val):
    global minpix
    minpix = val

def on_margin_sa(val):
    global margin_sa
    margin_sa = val

def on_moving_avg_max_count(val):
    global moving_avg_max_count
    global leftt_fit_avg_arr
    global right_fit_avg_arr

    moving_avg_max_count = val
    # Lanes polynomial Fit variables array
    if len(left_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(left_fit_avg_arr)):
            left_fit_avg_arr.append([])
    if len(right_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(right_fit_avg_arr)):
            right_fit_avg_arr.append([])

 
cv2.createTrackbar("nwindows","Polynomial Fit",nwindows,50,on_nwindows)
cv2.createTrackbar("margin","Polynomial Fit",margin,200,on_margin)
cv2.createTrackbar("minpix","Polynomial Fit",minpix,200,on_minpix)
cv2.createTrackbar("margin_sa","Polynomial Fit",margin_sa,640,on_margin_sa)
cv2.createTrackbar("moving_avg_len","Polynomial Fit",moving_avg_max_count,60,on_moving_avg_max_count)

while True:
    img_list=[]

    # Read image of current index
    img = cv2.imread(test_images[test_images_index])

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")[0]

    # Apply perspective transform to the undistorted thresholded image
    img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

    left_fit_avg_arr_filtered, right_fit_avg_arr_filtered = get_fit_avg_arr_filtered()

    # If both lane lines fit array arent empty, then search around
    if len(left_fit_avg_arr_filtered) > 0 and len(right_fit_avg_arr_filtered) > 0 :
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_search_around_poly(img_warped, margin_sa=margin_sa)
    else:
        # Calculate the Sliding window method to find lane lines 
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

    img_list.append(img)
    img_list.append(img_undistorted)
    img_list.append(img_und_thresholded)
    img_list.append(img_warped)
    img_list.append(img_warped_poly_fit)


    title_list = ["[{}]Original".format(test_images[test_images_index].split('/')[1]),"Undistorted","Combined Threshold","Warped","Polynomial Fit"]

    img_compostion = compose_image_arr(img_list,3,title_list=title_list,resize_factor=0.40)

    cv2.imshow("Polynomial Fit",img_compostion)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('e'):
        test_images_index += 1
        if(test_images_index >= len(test_images)):
            test_images_index = 0
    elif key == ord('w'):
        test_images_index -= 1
        if(test_images_index < 0):
            test_images_index = len(test_images)-1
    elif key == ord('s'):
        cv2.imwrite("output_images/image_thresholds_cv2.png",img_compostion)

cv2.destroyAllWindows()
print("Calibration Results:\n")

print("# Color and Gradient Threshold:")
print("  hls_threshold: ({},{})".format(hls_threshold[0],hls_threshold[1]))
print("  sobel_threshold: ({},{})".format(sobel_threshold[0],sobel_threshold[1]))

print("# Warp perspective:")
print("  dst_horizontal_offset = {}".format(dst_horizontal_offset))

print("# Sliding Window hyperparams:")
print("  nwindows = {}".format(nwindows))
print("  margin = {}".format(margin))
print("  minpix = {}".format(minpix))

print("# Search Around hyperparams:")
print("  margin_sa = {}".format(margin_sa))

print("# moving Average max length:")
print("  moving_avg_max_count = {}".format(moving_avg_max_count))

Calibration Results:

# Color and Gradient Threshold:
  hls_threshold: (160,255)
  sobel_threshold: (15,100)
# Warp perspective:
  dst_horizontal_offset = 360
# Sliding Window hyperparams:
  nwindows = 9
  margin = 100
  minpix = 50
# Search Around hyperparams:
  margin_sa = 100
# moving Average max length:
  moving_avg_max_count = 5


## 5. Determine the curvature of the lane and vehicle position with respect to center.

In [27]:
# Define conversions in x and y from pixels space to meters
ym_per_px = 30/720 # meters per pixel in y dimension
xm_per_px = 3.7/700 # meters per pixel in x dimension

In [29]:
def cal_lane_curvature(fity, left_fit, right_fit, mx=xm_per_px, my=ym_per_px, cal_meters=False):
    
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(fity)

    if(cal_meters):
        left_fit = np.array([left_fit[0]*mx/(my**2), left_fit[1]*mx/my, left_fit[2]*mx])
        right_fit = np.array([right_fit[0]*mx/(my**2), right_fit[1]*mx/my, right_fit[2]*mx])

    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

In [37]:
# change to next image index and reset moving average
test_images_index += 1 if test_images_index < (len(test_images)-1) else -(len(test_images)-1)
reset_ma()

## 5.1 Dynamic test of curvature values on different images

Instructions:
 - Press any key different than those described bellow to do a new loop iteration - This is needed because we are using a moving average to fit poly values, so for example, press `r` until fit line stabilizes.
 - `w` and `e` switch between images in folder `test_images`.
 - `s` saves image shown to folder `output_images`.
 - `q` exits simulation.

In [43]:
# Make a list of test images
test_images = sorted(glob.glob('test_images/*.jpg'))
test_images_index = 0
reset_ma()

while True:
    # Read image of current index
    img = cv2.imread(test_images[test_images_index])

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")[0]

    # Apply perspective transform to the undistorted thresholded image
    img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

    left_fit_avg_arr_filtered, right_fit_avg_arr_filtered = get_fit_avg_arr_filtered()

    # If both lane lines fit array arent empty, then search around
    if len(left_fit_avg_arr_filtered) > 0 and len(right_fit_avg_arr_filtered) > 0 :
        left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out = cal_search_around_poly(img_warped, margin_sa=margin_sa)
    else:
        # Calculate the Sliding window method to find lane lines 
        left_fit, right_fit, left_fitx, right_fitx, fity, img_binary_out = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

    dist_lines = right_fitx[-1] - left_fitx[-1]

    ym_per_px = 30/720 # meters per pixel in y dimension
    xm_per_px = 3.7/dist_lines # meters per pixel in x dimension
    
    left_curverad, right_curverad = cal_lane_curvature(fity,left_fit,right_fit)

    cv2.putText(img_binary_out,"Distance between lane lines: {:0.2f}px".format(dist_lines),(10,40),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,0,255),3,cv2.LINE_AA)
    cv2.putText(img_binary_out,"Left Radius: {:0.2f} \n Right Radius: {:0.2f}".format(left_curverad, right_curverad),(10,80),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,0,255),3,cv2.LINE_AA)

    print("\nInput Image: {}".format(test_images[test_images_index].split('/')[1]))
    print("Distance between lane lines: {:0.2f}px".format(dist_lines))
    print("Left Radius: {:0.2f} \n Right Radius: {:0.2f}".format(left_curverad, right_curverad))

    cv2.imshow("Calculate Curvature", img_binary_out)

    key = cv2.waitKey() & 0xFF
    if key == ord('q'):
        break
    elif key == ord('e'):
        test_images_index += 1
        if(test_images_index >= len(test_images)):
            test_images_index = 0
    elif key == ord('w'):
        test_images_index -= 1
        if(test_images_index < 0):
            test_images_index = len(test_images)-1
    elif key == ord('s'):
        cv2.imwrite("output_images/cal_curvature_{}.png".format(test_images[test_images_index].split('/')[1].split('.')[0]),img_binary_out)

cv2.destroyAllWindows()

ween lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radius: 1276.6495317275687
Distance between lane lines: 706.3950173881227px
Left Radius: 6665.424068343509 
 Right Radi

## 6. Warp the detected lane boundaries back onto the original image

730.06620671

## 7. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

# Dynamic Video Calibration - Full Project

In [178]:

# Calibrated Parameters

# Color and Gradient Threshold:
hls_threshold: (160,255)
sobel_threshold: (15,100)
# Warp perspective:
dst_horizontal_offset = 360
# Sliding Window hyperparams:
nwindows = 9
margin = 100
minpix = 50
# Search Around hyperparams:
margin_sa = 100
# moving Average max length:
moving_avg_max_count = 10

# Reset lanes polynomial fit average arrays
left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

In [175]:
# Reset lanes polynomial fit average arrays
def reset_ma():
    global left_fit_avg_arr
    global right_fit_avg_arr
    left_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]
    right_fit_avg_arr = [ [] for i in range(moving_avg_max_count)]

In [177]:
# Make a list of test videos
test_videos = sorted(glob.glob('*.mp4'))
test_videos_index = len(test_videos)-1
print(test_videos)

# Create a window with trackbars and callback functions bolow to handle when values are changed
cv2_window_name = "Advanced Lane Finding"
cv2.namedWindow(cv2_window_name)

# Threshold trackbars
def on_hls_th_l(val):
    global hls_threshold
    hls_threshold = (val,hls_threshold[1])

def on_hls_th_u(val):
    global hls_threshold
    hls_threshold = (hls_threshold[0],val)

def on_sobel_th_l(val):
    global sobel_threshold
    sobel_threshold = (val,sobel_threshold[1])

def on_sobel_th_u(val):
    global sobel_threshold
    sobel_threshold = (sobel_threshold[0],val)

cv2.createTrackbar("S Channel Threshold Lower",cv2_window_name,hls_threshold[0],255,on_hls_th_l)
cv2.createTrackbar("S Channel Threshold Upper",cv2_window_name,hls_threshold[1],255,on_hls_th_u)
cv2.createTrackbar("Sobel Threshold Lower",cv2_window_name,sobel_threshold[0],255,on_sobel_th_l)
cv2.createTrackbar("Sobel Threshold Upper",cv2_window_name,sobel_threshold[1],255,on_sobel_th_u)

# Warp perspective Trackbar
def on_warp_dst_x_offset(val):
    global dst_vertices
    global dst_horizontal_offset
    dst_horizontal_offset = val

    # Redefine destination vertices to warp
    dst_vertices = np.array(
        [[src_horizontal_center_x - dst_horizontal_offset, img_undistorted.shape[0]],
            [src_horizontal_center_x - dst_horizontal_offset, 0],
            [src_horizontal_center_x + dst_horizontal_offset, 0], 
            [src_horizontal_center_x + dst_horizontal_offset, img_undistorted.shape[0]]],
            np.float32)

cv2.createTrackbar("Dst Horizontal Offset",cv2_window_name,dst_horizontal_offset,600,on_warp_dst_x_offset)

# Polinomial Fit Trackbars
def on_nwindows(val):
    global nwindows
    nwindows = val

def on_margin(val):
    global margin
    margin = val

def on_minpix(val):
    global minpix
    minpix = val

def on_margin_sa(val):
    global margin_sa
    margin_sa = val

def on_moving_avg_max_count(val):
    global moving_avg_max_count
    global leftt_fit_avg_arr
    global right_fit_avg_arr

    moving_avg_max_count = val
    # Lanes polynomial Fit variables array
    if len(left_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(left_fit_avg_arr)):
            left_fit_avg_arr.append([])
    if len(right_fit_avg_arr) < moving_avg_max_count:
        for i in range(moving_avg_max_count - len(right_fit_avg_arr)):
            right_fit_avg_arr.append([])

 
cv2.createTrackbar("nwindows",cv2_window_name,nwindows,50,on_nwindows)
cv2.createTrackbar("margin",cv2_window_name,margin,200,on_margin)
cv2.createTrackbar("minpix",cv2_window_name,minpix,200,on_minpix)
cv2.createTrackbar("margin_sa",cv2_window_name,margin_sa,640,on_margin_sa)
cv2.createTrackbar("moving_avg_len",cv2_window_name,moving_avg_max_count,60,on_moving_avg_max_count)

while True:
    img_list=[]

    # Read a frame from video of current index
    ret,img = cap.read()

    if ret == False:
        cap = cv2.VideoCapture(test_videos[test_videos_index])
        continue

    # Undistort image
    img_undistorted = cal_undistort(img, mtx, dist)

    img_und_thresholded = cal_threshold(img_undistorted,hls_threshold=hls_threshold,sobel_threshold=sobel_threshold,color_space="BGR")[0]

    # Apply perspective transform to the undistorted thresholded image
    img_warped = cal_perspective(img_und_thresholded,src_vertices,dst_vertices)

    left_fit_avg_arr_filtered, right_fit_avg_arr_filtered = get_fit_avg_arr_filtered()

    # If both lane lines fit array arent empty, then search around
    if len(left_fit_avg_arr_filtered) > 0 and len(right_fit_avg_arr_filtered) > 0 :
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_search_around_poly(img_warped, margin_sa=margin_sa)
    else:
        # Calculate the Sliding window method to find lane lines 
        left_fit, right_fit, left_fitx, right_fitx, fity, img_warped_poly_fit = cal_sliding_window(img_warped,nwindows=nwindows,margin=margin,minpix=minpix)

    img_list.append(img)
    img_list.append(img_undistorted)
    img_list.append(img_und_thresholded)
    img_list.append(img_warped)
    img_list.append(img_warped_poly_fit)


    title_list = ["[{}]Original".format(test_videos[test_videos_index].split('.')[0]),"Undistorted","Combined Threshold","Warped","Polynomial Fit"]

    img_compostion = compose_image_arr(img_list,3,title_list=title_list,resize_factor=0.40)

    cv2.imshow(cv2_window_name,img_compostion)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        reset_ma()
    elif key == ord('e'):
        test_videos_index += 1
        if(test_videos_index >= len(test_videos)):
            test_videos_index = 0
        cap = cv2.VideoCapture(test_videos[test_videos_index])
        reset_ma()
    elif key == ord('w'):
        test_videos_index -= 1
        if(test_videos_index < 0):
            test_videos_index = len(test_videos)-1
        cap = cv2.VideoCapture(test_videos[test_videos_index])
        reset_ma()
    elif key == ord('s'):
        cv2.imwrite("output_images/video_composition_{}.png".format(test_videos[test_videos_index].split('.')[0]),img_compostion)

cv2.destroyAllWindows()
print("Calibration Results:")

print("\n# Color and Gradient Threshold:")
print("  hls_threshold: ({},{})".format(hls_threshold[0],hls_threshold[1]))
print("  sobel_threshold: ({},{})".format(sobel_threshold[0],sobel_threshold[1]))

print("\n# Warp perspective:")
print("  dst_horizontal_offset = {}".format(dst_horizontal_offset))

print("\n# Sliding Window hyperparams:")
print("  nwindows = {}".format(nwindows))
print("  margin = {}".format(margin))
print("  minpix = {}".format(minpix))

print("\n# Search Around hyperparams:")
print("  margin_sa = {}".format(margin_sa))

print("\n# moving Average max length:")
print("  moving_avg_max_count = {}".format(moving_avg_max_count))

['challenge_video.mp4', 'harder_challenge_video.mp4', 'project_video.mp4']
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
/home/renato/.virtualenvs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:41: RankWarning: Polyfit may be poorly conditioned
Calibration Results:

# Color and Gradient Threshold:
  hls_threshold: (127,253)
  sobel_threshold: (19,105)

# Warp perspective:
  dst_horizonta